In [2]:
!pip install earthengine-api

     ------------------------------------ 453.7/453.7 kB 525.5 kB/s eta 0:00:00
     -------------------------------------- 12.2/12.2 MB 198.7 kB/s eta 0:00:00
  Using cached google_cloud_storage-2.18.2-py2.py3-none-any.whl (130 kB)
     -------------------------------------- 96.9/96.9 kB 395.5 kB/s eta 0:00:00
     ------------------------------------ 139.4/139.4 kB 330.4 kB/s eta 0:00:00
     -------------------------------------- 81.3/81.3 kB 268.0 kB/s eta 0:00:00
     ------------------------------------ 200.9/200.9 kB 369.4 kB/s eta 0:00:00
     -------------------------------------- 50.1/50.1 kB 510.8 kB/s eta 0:00:00
     ------------------------------------ 220.9/220.9 kB 435.2 kB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.23.4
    Uninstalling google-auth-2.23.4:
      Successfully uninstalled google-auth-2.23.4


In [4]:
!pip install geemap


     ---------------------------------------- 2.2/2.2 MB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 665.3 kB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 642.6 kB/s eta 0:00:00
  Using cached ipyleaflet-0.19.2-py3-none-any.whl (31 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached folium-0.17.0-py2.py3-none-any.whl (108 kB)
     ---------------------------------------- 1.2/1.2 MB 131.3 kB/s eta 0:00:00
  Using cached scooby-0.10.0-py3-none-any.whl (18 kB)
     -------------------------------------- 98.6/98.6 kB 161.5 kB/s eta 0:00:00
     ------------------------------------ 101.8/101.8 kB 244.1 kB/s eta 0:00:00
  Using cached xyzservices-2024.6.0-py3-none-any.whl (83 kB)
  Using cached branca-0.7.2-py3-none-any.whl (25 kB)
  Using cached traittypes-0.2.1-py2.py3-none-any.whl (8.6 kB)
  Using cached jupyter_leaflet-0.19.2-py3-none-any.whl (1.1 MB)
  Using c

In [5]:
import ee
import geemap.core as geemap

In [6]:
ee.Authenticate()




Successfully saved authorization token.


In [7]:

ee.Initialize(project='ee-gdaproject')

In [11]:
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


In [10]:
# Daily mean 2m air temperature
era5_2mt = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('mean_2m_air_temperature')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)
display(era5_2mt)

# Daily total precipitation sums
era5_tp = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('total_precipitation')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)

# Daily mean 2m dewpoint temperature
era5_2d = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('dewpoint_2m_temperature')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)

# Daily mean sea-level pressure
era5_mslp = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('mean_sea_level_pressure')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)

# Daily mean surface pressure
era5_sp = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('surface_pressure')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)

# Daily mean 10m u-component of wind
era5_u_wind_10m = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('u_component_of_wind_10m')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)

# Convert pressure levels from Pa to hPa - Example for surface pressure
era5_sp = era5_sp.map(
    lambda image: image.divide(100).set(
        'system:time_start', image.get('system:time_start')
    )
)

# Visualization palette for total precipitation
vis_tp = {
    'min': 0.0,
    'max': 0.1,
    'palette': ['ffffff', '00ffff', '0080ff', 'da00ff', 'ffa400', 'ff0000'],
}

# Visualization palette for temperature (mean, min and max) and 2m dewpoint
# temperature
vis_2mt = {
    'min': 250,
    'max': 320,
    'palette': [
        '000080',
        '0000d9',
        '4000ff',
        '8000ff',
        '0080ff',
        '00ffff',
        '00ff80',
        '80ff00',
        'daff00',
        'ffff00',
        'fff500',
        'ffda00',
        'ffb000',
        'ffa400',
        'ff4f00',
        'ff2500',
        'ff0a00',
        'ff00ff',
    ],
}

# Visualization palette for u- and v-component of 10m wind
vis_wind = {
    'min': 0,
    'max': 30,
    'palette': [
        'ffffff',
        'ffff71',
        'deff00',
        '9eff00',
        '77b038',
        '007e55',
        '005f51',
        '004b51',
        '013a7b',
        '023aad',
    ],
}

# Visualization palette for pressure (surface pressure, mean sea level
# pressure) - adjust min and max values for mslp to 'min':990 and 'max':1050
vis_pressure = {
    'min': 500,
    'max': 1150,
    'palette': [
        '01ffff',
        '058bff',
        '0600ff',
        'df00ff',
        'ff00ff',
        'ff8c00',
        'ff8c00',
    ],
}


# Add layer to map
m = geemap.Map()
m.add_layer(
    era5_tp.filter(ee.Filter.date('2019-07-15')),
    vis_tp,
    'Daily total precipitation sums',
)
m.add_layer(
    era5_2d.filter(ee.Filter.date('2019-07-15')),
    vis_2mt,
    'Daily mean 2m dewpoint temperature',
)
m.add_layer(
    era5_2mt.filter(ee.Filter.date('2019-07-15')),
    vis_2mt,
    'Daily mean 2m air temperature',
)
m.add_layer(
    era5_u_wind_10m.filter(ee.Filter.date('2019-07-15')),
    vis_wind,
    'Daily mean 10m u-component of wind',
)
m.add_layer(
    era5_sp.filter(ee.Filter.date('2019-07-15')),
    vis_pressure,
    'Daily mean surface pressure',
)

m.set_center(21.2, 22.2, 2)
m

Map(center=[22.2, 21.2], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…